In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model
import pickle


data = pd.read_csv('credit_card_transactions.csv')
data = data.drop('transaction_id', axis=1)
data = data.drop('receiver_expiry_date', axis=1)
data = data.drop('receiver_CVV/CVC', axis=1)
data = data.drop('receiver_bank_name', axis=1)
data = data.drop('sender_bank_name', axis=1)
data = data.drop('time', axis=1)
data = data.drop('sender_expiry_date', axis=1)

In [2]:
# # Label encoding
# from sklearn.preprocessing import LabelEncoder
# le_sender = LabelEncoder()
# data["sender_bank_name"] = le_sender.fit_transform(data["sender_bank_name"])

# le_receiver = LabelEncoder()
# data["receiver_bank_name"] = le_receiver.fit_transform(data["receiver_bank_name"])

# le_sender_expiry = LabelEncoder()
# data["sender_expiry_date"] = le_sender_expiry.fit_transform(data["sender_expiry_date"])

# le_sender_cvv = LabelEncoder()
# data["sender_CVV/CVC"] = le_sender_cvv.fit_transform(data["sender_CVV/CVC"])

In [3]:
X = data.drop('is_fraudulent', axis=1)
y = data['is_fraudulent']

In [4]:
X.head();

print(X);

        sender_card_number  sender_CVV/CVC  receiver_card_number  \
0         7807007958935238             143      7419312478830388   
1         6166774269071520             707      6238064142244445   
2         6940438243386529             544      8435150293912320   
3         7190279905168940             224      7720830884824111   
4         6108360084817868             338      7842057686854777   
...                    ...             ...                   ...   
999995    7651827082411425             364      9134862526205151   
999996    9474874213570467             359      6337993645142928   
999997    8310594043608541             237      9388585067345151   
999998    7588059459931859             762      6733220873933616   
999999    9987578700177858             983      9364612464546104   

        transfer_amount  
0            913.353029  
1            299.377917  
2            308.881622  
3            791.305051  
4            158.730742  
...                 ...  
9

In [5]:
print(y)

0         0
1         1
2         0
3         1
4         1
         ..
999995    0
999996    1
999997    0
999998    0
999999    0
Name: is_fraudulent, Length: 1000000, dtype: int64


In [6]:
X=X.dropna()
y=y.dropna()
print(X)
print(y)

        sender_card_number  sender_CVV/CVC  receiver_card_number  \
0         7807007958935238             143      7419312478830388   
1         6166774269071520             707      6238064142244445   
2         6940438243386529             544      8435150293912320   
3         7190279905168940             224      7720830884824111   
4         6108360084817868             338      7842057686854777   
...                    ...             ...                   ...   
999995    7651827082411425             364      9134862526205151   
999996    9474874213570467             359      6337993645142928   
999997    8310594043608541             237      9388585067345151   
999998    7588059459931859             762      6733220873933616   
999999    9987578700177858             983      9364612464546104   

        transfer_amount  
0            913.353029  
1            299.377917  
2            308.881622  
3            791.305051  
4            158.730742  
...                 ...  
9

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=39)

In [8]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(X_train_scaled)

[[-0.08473204  1.58174044  0.01554939  1.45644268]
 [ 0.42217235 -1.53335608  0.16045704  0.60996741]
 [ 0.75191193 -1.34830084 -1.48653176 -1.41058024]
 ...
 [-0.93207932 -0.63892243  0.43566404  0.28098612]
 [ 1.31617808 -1.43311783 -0.96367056 -0.32277152]
 [-1.14095814 -0.5888033  -1.2223218  -1.06820399]]


In [9]:
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_model.fit(X_train, y_train)
y_pred_rf = random_forest_model.predict(X_test)

In [10]:
# Save the Random Forest model
with open('random_forest_model.pkl', 'wb') as file:
    pickle.dump(random_forest_model, file)

In [11]:
neural_network_model = Sequential()
neural_network_model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
neural_network_model.add(Dense(1, activation='sigmoid'))
neural_network_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
neural_network_model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=1)
pred_x=neural_network_model.predict(X_test);
y_pred_nn=np.argmax(pred_x,axis=1);

Epoch 1/5
12500/12500 [==============================] - 26s 2ms/step - loss: 5824723812352.0000 - accuracy: 0.4998
Epoch 2/5
12500/12500 [==============================] - 25s 2ms/step - loss: 4636880142336.0000 - accuracy: 0.4996
Epoch 3/5
12500/12500 [==============================] - 24s 2ms/step - loss: 4120031002624.0000 - accuracy: 0.5000
Epoch 4/5
12500/12500 [==============================] - 25s 2ms/step - loss: 3610479427584.0000 - accuracy: 0.4999
Epoch 5/5
6250/6250 [==============================] - 10s 2ms/step


In [12]:
# Save the Neural Network model
# with open('neural_network_model.pkl', 'wb') as file:
#     pickle.dump(neural_network_model, file)
neural_network_model.save('neural_network_model.h5')

In [13]:
y_pred_combined = np.round((y_pred_rf + y_pred_nn) / 2)

In [14]:
accuracy = accuracy_score(y_test, y_pred_combined)
conf_matrix = confusion_matrix(y_test, y_pred_combined)
classification_rep = classification_report(y_test, y_pred_combined)

c:\Users\vishw\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vishw\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vishw\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [15]:
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')


Accuracy: 0.499095
Confusion Matrix:
[[ 99819      0]
 [100181      0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.50      1.00      0.67     99819
           1       0.00      0.00      0.00    100181

    accuracy                           0.50    200000
   macro avg       0.25      0.50      0.33    200000
weighted avg       0.25      0.50      0.33    200000

